### TraVeL GAN

In [1]:
import pathlib
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from tqdm import tqdm
from PIL import Image
import os
from torch.optim.lr_scheduler import ReduceLROnPlateau
import joblib

In [2]:
ngpu = 0
pathA = "Data/SpectrogramsA"
pathB = "Data/SpectrogramsB"
filename_gen = "TraVeLGAN/no_smoothing_with_noise_gen4layers_20epoci_new/TraVeLGAN-Gen.joblib"
filename_dis = "TraVeLGAN/no_smoothing_with_noise_gen4layers_20epoci_new/TraVeLGAN-Dis.joblib"
filename_sim = "TraVeLGAN/no_smoothing_with_noise_gen4layers_20epoci_new/TraVeLGAN-Siam.joblib"
workers = 2
batch_size = 32

image_size = 32

# channels nr for input images
nc = 1
# size of feature maps in generator
ngf = 64
# size of feature maps in discriminator
ndf = 64
# length of latent vector
lenvec = 128

num_epochs = 20
lr = 0.0002
lrD = 0.000001
beta1 = 0.5

def find_classes(directory:str):
    classes = sorted(entry.name for entry in os.scandir(directory) if entry.is_dir())

    class_to_index = {class_name: i for i, class_name in enumerate(classes)}

    return classes, class_to_index

In [3]:
class MyImageFolder(torch.utils.data.Dataset):
    def __init__(self, directory:str, transform=None):

        self.paths = list(pathlib.Path(directory).glob('*/*.jpg')) * 3

        self.transform = transform
        self.classes, self.class_to_index = find_classes(directory)

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        img = self.load_image(index)
        class_name = self.paths[index].parent.name
        class_index = self.class_to_index[class_name]

        if self.transform:
            return self.transform(img), class_index
        else:
            return img, class_index

    def load_image(self, index):
        image_path = self.paths[index]
        return Image.open(image_path)

transform = torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),  # imaginile sunt normalizate intre [0, 1]
            torchvision.transforms.Normalize(0.5, 0.5),  # imaginile sunt normalizate intre [-1, 1]
        ])


In [4]:
train_data_A = MyImageFolder(directory=pathA, transform=transform)
train_data_B = MyImageFolder(directory=pathB, transform=transform)

dataloader_A = torch.utils.data.DataLoader(train_data_A, batch_size=batch_size, shuffle=True, num_workers=2)
dataloader_B = torch.utils.data.DataLoader(train_data_B, batch_size=batch_size, shuffle=True, num_workers=2)
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")


In [5]:
'''
Gaussian distribution (also known as normal distribution) is a bell-shaped curve, and it is assumed that during any measurement values will follow a normal distribution with an equal number of measurements above and below the mean value.
'''
class GaussianNoise(torch.nn.Module):
    def __init__(self, std=0.1, decay=0):
        super().__init__()
        self.std = std
        self.decay = decay

    def step(self):
        self.std = max(self.std - self.decay, 0)

    def forward(self, x):
        if self.training:
            return x + torch.empty_like(x).normal_(std=self.std)
        else:
            return x

In [6]:
'''
From the DCGAN paper, the authors specify that all model weights shall be randomly initialized from a Normal distribution with mean=0, stdev=0.02. The weights_init function takes an initialized model as input and reinitializes all convolutional, convolutional-transpose, and batch normalization layers to meet this criteria. This function is applied to the models immediately after initialization.
'''
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0)

In [7]:
generator = torch.nn.Sequential(

    # downsampling
    torch.nn.Conv2d(nc, ngf, kernel_size=4, stride=2, padding=1, bias=False),
    torch.nn.BatchNorm2d(ngf),
    torch.nn.LeakyReLU(True),

    torch.nn.Conv2d(ngf, ngf * 2, kernel_size=4, stride=2, padding=1, bias=False),
    torch.nn.BatchNorm2d(ngf * 2),
    torch.nn.LeakyReLU(True),

    # upsampling
    torch.nn.ConvTranspose2d(ngf * 2, ngf, kernel_size=4, stride=2, padding=1, bias=False),
    torch.nn.BatchNorm2d(ngf),
    torch.nn.LeakyReLU(True),

    torch.nn.ConvTranspose2d(ngf, nc, kernel_size=4, stride=2, padding=1, bias=False),
    torch.nn.Tanh(),
    # nc x 32 x 16
)

generator.apply(weights_init)




Sequential(
  (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): LeakyReLU(negative_slope=True)
  (3): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): LeakyReLU(negative_slope=True)
  (6): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): LeakyReLU(negative_slope=True)
  (9): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (10): Tanh()
)

In [8]:
# Siamese model
siamese = torch.nn.Sequential(
    # 1 x 32 x 16
    torch.nn.Conv2d(nc, ngf, kernel_size=4, stride=2, padding=1, bias=False),
    torch.nn.BatchNorm2d(ngf),
    torch.nn.LeakyReLU(True),
    # ngf x 16 x 8

    torch.nn.Conv2d(ngf, ngf, kernel_size=(3, 3), stride=(2, 1), padding=(1, 1), bias=False),
    torch.nn.BatchNorm2d(ngf),
    torch.nn.LeakyReLU(True),
    # ngf x 8 x 8

    torch.nn.Conv2d(ngf, ngf * 2, kernel_size=(3, 3), stride=(1, 2), padding=(1, 1), bias=False),
    torch.nn.BatchNorm2d(ngf * 2),
    torch.nn.LeakyReLU(True),
    # ngf*2 x 8 x 4

    torch.nn.Flatten(),
    # ngf*2*8*4
    torch.nn.Linear(ngf * 64, lenvec)
)

In [9]:
# Discriminator model
discriminator = torch.nn.Sequential(
    # nc x 32 x 32
    GaussianNoise(),
    torch.nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
    torch.nn.BatchNorm2d(ndf),
    torch.nn.LeakyReLU(0.2, inplace=True),
    # ndf x 16 x 16

    GaussianNoise(),
    torch.nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
    torch.nn.BatchNorm2d(ndf * 2),
    torch.nn.LeakyReLU(0.2, inplace=True),
    # ndf*2 x 8 x 8

    GaussianNoise(),
    torch.nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
    torch.nn.BatchNorm2d(ndf * 4),
    torch.nn.LeakyReLU(0.2, inplace=True),
    # ndf*4 x 4 x 4

    GaussianNoise(),
    torch.nn.Conv2d(ndf * 4, 1, 4, 1, 0, bias=False),
    torch.nn.Sigmoid(),
    # 1 x 1 x 1
)
discriminator.apply(weights_init)



Sequential(
  (0): GaussianNoise()
  (1): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): LeakyReLU(negative_slope=0.2, inplace=True)
  (4): GaussianNoise()
  (5): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): LeakyReLU(negative_slope=0.2, inplace=True)
  (8): GaussianNoise()
  (9): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (10): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): LeakyReLU(negative_slope=0.2, inplace=True)
  (12): GaussianNoise()
  (13): Conv2d(256, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
  (14): Sigmoid()
)

In [10]:
class IdentityLoss(torch.nn.Module):
    def __init__(self):
        super(IdentityLoss, self).__init__()

    def forward(self, real, generated):
        identity_loss = torch.nn.functional.l1_loss(real, generated)
        return identity_loss

class CosineSimilarityLoss(torch.nn.Module):
    def __init__(self):
        super(CosineSimilarityLoss, self).__init__()

    def forward(self, a, b):
        a_normalized = torch.nn.functional.normalize(a, dim=1)
        b_normalized = torch.nn.functional.normalize(b, dim=1)
        cos_sim = torch.mean(torch.nn.functional.cosine_similarity(a_normalized, b_normalized, dim=1))
        return cos_sim

class SiameseLoss(torch.nn.Module):
    def __init__(self, margin):
        super(SiameseLoss, self).__init__()
        self.margin = margin

    def forward(self, a1, a2):
        distances = torch.sqrt(torch.sum((a1 - a2) ** 2, dim=-1, keepdim=True))
        hinge_loss = torch.square(torch.maximum((self.margin - distances), torch.tensor(0)))
        loss = torch.mean(hinge_loss)
        return loss

In [11]:
adversarial_loss = torch.nn.BCELoss()
identity_loss = IdentityLoss()
cossine_loss = CosineSimilarityLoss()
siamese_loss = SiameseLoss(1)

fixed_noise = torch.randn(batch_size, nc, image_size, image_size, device=device)
different_classes = -1
same_class = 1
real_label = 0
fake_label = 1

gen_optimizer = torch.optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))
dis_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lrD, betas=(beta1, 0.999))
scheduler_gen = ReduceLROnPlateau(gen_optimizer, patience=1, factor=0.1, min_lr=0.00000001, verbose=True)
#scheduler_dis = ReduceLROnPlateau(dis_optimizer, patience=1, factor=0.1, min_lr=0.0000001)

In [12]:
def normalize(x, minim, maxim):
    return (x - minim) / (maxim - minim)

# def smooth_label(label):
#     arr = torch.rand((b_size,), dtype=torch.float, device=device)
#     if label == 1:
#         for i in range(arr.shape[0]):
#             arr[i] = normalize(arr[i], 0.7, 1.2)
#     else:
#         for i in range(arr.shape[0]):
#             arr[i] = normalize(arr[i], 0.0, 0.3)
#     return arr

def slice_images(batch):
    part1 = batch[:, :, :, :batch.shape[3] // 2].clone()
    part2 = batch[:, :, :, batch.shape[3] // 2 :].clone()
    return part1, part2

def assemble_images(part1, part2):
    image = torch.cat([part1, part2], dim=3)
    return image

img_lists = []
gen_losses = []
dis_losses = []
iters = 0


In [13]:
def traind_discriminator(A, B):
    A1, A2 = slice_images(A)
    A_b_size = A1.size(0)
    B_b_size = B.size(0)
    gen_A1 = generator(A1)
    gen_A2 = generator(A2)
    AB = assemble_images(gen_A1, gen_A2)
    output_AB = discriminator(AB.detach()).view(-1)
    output_B = discriminator(B).view(-1)

    label_fake = torch.full((B_b_size,), fake_label, dtype=torch.float, device=device)
    zero = torch.full((B_b_size,), 0, dtype=torch.float, device=device)
    d_loss_real = torch.mean(torch.max(label_fake + output_B, zero))

    zero = torch.full((A_b_size,), 0, dtype=torch.float, device=device)
    label_f = torch.full((A_b_size,), fake_label, dtype=torch.float, device=device)
    d_loss_fake = torch.mean(torch.max(label_f - output_AB, zero))

    d_total_loss = (d_loss_real + d_loss_fake) / 2

    dis_optimizer.zero_grad()
    d_total_loss.backward()
    dis_optimizer.step()

    return d_loss_real, d_loss_fake

In [ ]:
for epoch in range(num_epochs):
    g_total_loss = 0
    d_total_loss = 0
    dis_b = 0
    dis_ab = 0

    torch.autograd.set_detect_anomaly(True)
    for i, (data_A, data_B) in tqdm(enumerate(zip(dataloader_A, dataloader_B))):

        real_A = data_A[0].to(device)
        real_B = data_B[0].to(device)
        # data[0] sunt imaginile
        # data[1] sunt labelurile

        A1, A2 = slice_images(real_A)
        B1, B2 = slice_images(real_B)

        A_b_size = A1.size(0)  # batch size
        B_b_size = B1.size(0)

        if i % 1 == 0:

            # Training the discriminator
            gen_A1 = generator(A1)
            gen_A2 = generator(A2)
            gen_B1 = generator(B1)
            gen_B2 = generator(B2)

            # identity mapping loss
            id_loss = (identity_loss(B1, gen_B1) + identity_loss(B2, gen_B2)) / 2
            if i % 60 == 0: print(f"Id loss {id_loss:.4f}")

            AB = assemble_images(gen_A1, gen_A2)

            output_AB = discriminator(AB.detach()).view(-1)
            output_B = discriminator(real_B).view(-1)
            if i % 60 == 0: print(output_AB, output_B)


            label_fake = torch.full((B_b_size,), fake_label, dtype=torch.float, device=device)
            zero = torch.full((B_b_size,), 0, dtype=torch.float, device=device)
            d_loss_real = torch.mean(torch.max(label_fake + output_B, zero))
            if i % 60 == 0: print(f"D loss real: {d_loss_real:.4f}")

            label_real = torch.full((A_b_size,), real_label, dtype=torch.float, device=device)
            label_f = torch.full((A_b_size,), fake_label, dtype=torch.float, device=device)
            zero = torch.full((A_b_size,), 0, dtype=torch.float, device=device)
            #d_loss_fake = adversarial_loss(output_AB, label_fake)
            d_loss_fake = torch.mean(torch.max(label_f - output_AB, zero))
            if i % 60 == 0: print(f"D loss fake: {d_loss_fake:.4f}")

            # Train siamese
            siamese.zero_grad()
            siamese_AB1 = siamese(gen_A1)
            siamese_AB2 = siamese(gen_A2)
            siamese_A1 = siamese(A1)
            siamese_A2 = siamese(A2)


            t_loss = cossine_loss((siamese_A1 - siamese_A2), (siamese_AB1 - siamese_AB2))
            s_loss = siamese_loss(siamese_A1, siamese_A2)
            ts_loss = 3 - t_loss - s_loss

            if i % 60 == 0: print(f"Ts loss {ts_loss:.4f}")

            label_gen = torch.full((A_b_size,), fake_label, dtype=torch.float, device=device)
            disc = discriminator(AB.detach()).view(-1)
            g_loss = torch.mean(-(label_gen - disc))
            if i % 60 == 0: print(f"G loss :{g_loss:.4f}")

            d_total_loss = (d_loss_real + d_loss_fake) / 2
            g_total_loss = g_loss + 10. * ts_loss + 0.5 * id_loss

            dis_b = output_B.mean().item()
            dis_ab = output_AB.mean().item()

            gen_optimizer.zero_grad()
            g_total_loss.backward()
            gen_optimizer.step()

            dis_optimizer.zero_grad()
            d_total_loss.backward()
            dis_optimizer.step()

        else:
            d_loss_real, d_loss_fake = traind_discriminator(real_A, real_B)
            d_total_loss = (d_loss_real + d_loss_fake) / 2

        if i % 60 == 0:
            print(f"[{epoch}/{num_epochs - 1}] [{i}/{len(dataloader_A) - 1}]\tLoss_dis: {d_total_loss.item():.4f}\tLoss_gen: {g_total_loss.item():.4f}\tD(x): {dis_b:.4f}\tD(G(x)): {dis_ab:.4f}")

        gen_losses.append(g_total_loss.item())
        dis_losses.append(d_total_loss.item())

        # if iters % 500 == 0 or ((epoch == num_epochs - 1) and (i == len(dataloader) - 1)):
        with torch.no_grad():
            fake = generator(A1).detach().cpu()
        img_lists.append(torchvision.utils.make_grid(fake, padding=2, normalize=True))

        iters += 1
    scheduler_gen.step(g_total_loss.item())
    #scheduler_dis.step(d_total_loss.item())


    joblib.dump(generator, filename_gen)
    joblib.dump(discriminator, filename_dis)
    joblib.dump(siamese, filename_sim)

    gen_losses_arr = np.array(gen_losses, dtype=object)
    dis_losses_arr = np.array(dis_losses, dtype=object)
    img_arr = np.array(img_lists, dtype=object)
    np.save('TraVeLGAN/no_smoothing_with_noise_gen4layers_20epoci_new/TraVeLGAN-gen_losses.npy', gen_losses_arr)
    np.save('TraVeLGAN/no_smoothing_with_noise_gen4layers_20epoci_new/TraVeLGAN-dis_losses.npy', dis_losses_arr)
    np.save('TraVeLGAN/no_smoothing_with_noise_gen4layers_20epoci_new/TraVeLGAN-imgs.npy', img_arr)


1it [00:00,  4.53it/s]

Id loss 0.9858
tensor([0.5207, 0.5728, 0.7009, 0.6354, 0.8902, 0.4525, 0.5090, 0.4213, 0.6665,
        0.5486, 0.5599, 0.5575, 0.8462, 0.5609, 0.3520, 0.6421, 0.7544, 0.5796,
        0.8853, 0.6821, 0.8126, 0.4909, 0.5965, 0.7253, 0.6199, 0.6286, 0.8769,
        0.4683, 0.7523, 0.5877, 0.6040, 0.3608], grad_fn=<ViewBackward0>) tensor([0.5103, 0.5911, 0.5245, 0.5630, 0.6532, 0.6956, 0.9156, 0.5865, 0.4673,
        0.5896, 0.3059, 0.5617, 0.4784, 0.8172, 0.4938, 0.5593, 0.7253, 0.6825,
        0.5519, 0.5919, 0.5237, 0.3899, 0.5565, 0.5386, 0.5756, 0.4114, 0.6107,
        0.5001, 0.5463, 0.6147, 0.3624, 0.6468], grad_fn=<ViewBackward0>)
D loss real: 1.5669
D loss fake: 0.3793
Ts loss 2.9000
G loss :-0.4155
[0/19] [0/538]	Loss_dis: 0.9731	Loss_gen: 29.0769	D(x): 0.5669	D(G(x)): 0.6207


61it [00:11,  5.07it/s]

Id loss 0.9089
tensor([0.4786, 0.6756, 0.5469, 0.7599, 0.2737, 0.4949, 0.4892, 0.7286, 0.8557,
        0.6223, 0.5880, 0.7334, 0.6770, 0.9441, 0.9697, 0.5112, 0.7015, 0.5861,
        0.6989, 0.5865, 0.7231, 0.6911, 0.3041, 0.6438, 0.6063, 0.5020, 0.4868,
        0.5725, 0.5285, 0.6891, 0.4139, 0.6520], grad_fn=<ViewBackward0>) tensor([0.2484, 0.5665, 0.2932, 0.2475, 0.9261, 0.3473, 0.5203, 0.4995, 0.3089,
        0.6918, 0.4391, 0.6367, 0.4148, 0.3277, 0.3894, 0.3806, 0.1969, 0.3174,
        0.2242, 0.2975, 0.4414, 0.9187, 0.4420, 0.4826, 0.5398, 0.3948, 0.3597,
        0.3502, 0.5611, 0.9012, 0.4306, 0.3928], grad_fn=<ViewBackward0>)
D loss real: 1.4528
D loss fake: 0.3833
Ts loss 2.2255
G loss :-0.3789
[0/19] [60/538]	Loss_dis: 0.9180	Loss_gen: 22.3301	D(x): 0.4528	D(G(x)): 0.6167


121it [00:24,  4.75it/s]

...

480it [01:59,  2.99it/s]

Id loss 0.2631
tensor([0.9424, 0.9913, 0.9484, 0.9048, 0.9682, 0.9861, 0.9198, 0.9407, 0.9909,
        0.9703, 0.9192, 0.9514, 0.9238, 0.8755, 0.9051, 0.9338, 0.9906, 0.9516,
        0.8449, 0.9525, 0.9259, 0.9652, 0.9690, 0.9938, 0.9280, 0.9313, 0.9949,
        0.9331, 0.8794, 0.9428, 0.8509, 0.9164], grad_fn=<ViewBackward0>) tensor([0.3057, 0.0048, 0.2688, 0.3508, 0.0689, 0.2645, 0.1988, 0.3891, 0.3012,
        0.1726, 0.5088, 0.1885, 0.6877, 0.1658, 0.8259, 0.4641, 0.3340, 0.1940,
        0.2293, 0.2993, 0.2607, 0.1489, 0.3121, 0.3524, 0.1346, 0.1194, 0.0638,
        0.3140, 0.2010, 0.2063, 0.0554, 0.2683], grad_fn=<ViewBackward0>)
D loss real: 1.2706
D loss fake: 0.0612
Ts loss 1.8543
G loss :-0.0757


481it [02:00,  2.71it/s]

[8/19] [480/538]	Loss_dis: 0.6659	Loss_gen: 18.5988	D(x): 0.2706	D(G(x)): 0.9388


539it [02:16,  3.96it/s]


In [ ]:
generator = joblib.load(filename_gen)
# discriminator = joblib.load(filename_dis)

In [ ]:
test_spec = Image.open("Data/SpectrogramsA/pop/pop_spec1.jpg")
make_tensor = torchvision.transforms.ToTensor()
test_spec = transform(test_spec)
test_spec = torch.transpose(test_spec, 0, 2)

fig, axs = plt.subplots(1)
axs.imshow(test_spec)
plt.show()


test_spec = torch.transpose(test_spec, 0, 2)
test_spec = torch.unsqueeze(test_spec, 0)
# test_spec = torch.transpose(test_spec, 0, 1)

print(test_spec.shape)
test1, test2 = slice_images(test_spec)
test_spec1 = generator(test1)
test_spec2 = generator(test2)
test_spec = assemble_images(test_spec1, test_spec2)

test_spec = torch.squeeze(test_spec, 0)
test_spec = torch.transpose(test_spec, 0, 2)

print(test_spec.shape)
fig, axs = plt.subplots(1)
axs.imshow(test_spec.detach())
plt.show()



In [ ]:
rock_batch = next(iter(dataloader_A))
p = rock_batch[0]
res = generator(p)
#b1, b2 = slice_images(rock_batch)
# p1 = generator(b1)
# p2 = generator(b2)
# p = assemble_images(p1, p2)

plt.figure(figsize=(25,25))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Fake images")
plt.imshow(np.transpose(torchvision.utils.make_grid(res.to(device), padding=5, normalize=True).cpu(),(1,2,0)))



# plt.subplot()
# plt.axis("off")
# plt.title("Fake Images")
# plt.imshow(np.transpose(res.detach(),(1,2,0)))
# plt.show()

In [ ]:
# real_batch = next(iter(dataloader))
real_batch = next(iter(dataloader_A))

plt.figure(figsize=(25,25))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real images")
plt.imshow(np.transpose(torchvision.utils.make_grid(rock_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))


# plt.subplot(1,2,2)
# plt.axis("off")
# plt.title("Fake Images")
# plt.imshow(np.transpose(img_lists[-1],(1,2,0)))
# plt.show()